In [14]:
import http.server
import threading
import numpy as np
import urllib.request

# sys.path.append(r"C:\projects\OpenVisus\build22\RelWithDebInfo")
import OpenVisus as ov

In [5]:
urllib.request.urlretrieve('https://klacansky.com/open-scivis-datasets/hydrogen_atom/hydrogen_atom_128x128x128_uint8.raw', 'hydrogen_atom_128x128x128_uint8.raw')
data = np.fromfile('hydrogen_atom_128x128x128_uint8.raw', dtype=np.uint8).reshape(128, 128, 128)

In [6]:
import shutil
shutil.rmtree("./hydrogen_atom",ignore_errors=True) 
db = ov.CreateIdx(
  url="hydrogen_atom.idx", 
  dims=list(reversed(data.shape)), 
  fields=[ov.Field('data','uint8')], 
  arco='1mb')

# NOTE: this will create internally a DiskAccess (because there is 'arco' directive)
# DIsk access will create one-file per block. Meaning that you will have 2 files each of 1MB (128*128*128*sizeof(uint8)==2MiB)
#
# IMPORTANT: in writing mode the data is written UNCOMPRESSED to avoid multiple decompress/compress operations. 
# #          Remember to compress it before uploading to S3 or serving with -m http.server since the clients
#            will assume the data has been compressed
db.write(data)

In [7]:
# this step compress all blocks in ZIP (needed for read-client access!)
db.compressDataset()

In [9]:
# here you can start a server, or run from external doing `python3 -m http.server`
def run(PORT = 8000):
    Handler = http.server.SimpleHTTPRequestHandler
    with http.server.socketserver.TCPServer(("", PORT), Handler) as httpd:
        httpd.serve_forever()
threading.Thread(target=run).start()

In [11]:
# example of accessing the data using a simple HTTP server
# IMPORANT: internally db.read will call createAccess that will create a `CloudStorageAccess` 
db = ov.LoadDataset('http://localhost:8000/hydrogen_atom.idx')
data_ = db.read()

127.0.0.1 - - [28/Nov/2022 08:29:28] "GET /tmp/hydrogen_atom.idx HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 08:29:28] "GET /tmp/hydrogen_atom/0/data/0000/0000/0000/0001.bin HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 08:29:28] "GET /tmp/hydrogen_atom/0/data/0000/0000/0000/0000.bin HTTP/1.1" 200 -


In [12]:
assert (data == data_).all()